#### Recommender Another Example

**DataSet**

https://grouplens.org/datasets/movielens/

In [1]:
#import
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [4]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [5]:
data.head()

Row(movieId=2, rating=3, userId=0)

In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|                 1|                 0|
|    max|                99|                 5|                29|
+-------+------------------+------------------+------------------+



In [7]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [8]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

In [9]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [10]:
predictions.show()

+-------+------+------+------------+
|movieId|rating|userId|  prediction|
+-------+------+------+------------+
|     31|     1|    26|-0.007019192|
|     31|     1|    27| -0.67001355|
|     31|     1|     5|   2.7408295|
|     31|     1|    19|   1.5565219|
|     31|     1|    24|   1.8276801|
|     31|     1|    29|   1.8471841|
|     85|     1|    28|   3.5813441|
|     85|     1|    13|    3.354125|
|     85|     3|     6|   2.3255682|
|     65|     1|    19|-0.086809486|
|     65|     1|    24|   0.7722943|
|     53|     3|    20|   1.7325065|
|     53|     5|     8|   3.6216986|
|     53|     3|    14|  0.66858095|
|     78|     1|    27|   0.7415864|
|     78|     1|     1|   1.0368232|
|     78|     1|     2|  0.62639916|
|     34|     1|    15|   1.8475008|
|     34|     4|     2|   1.2207305|
|     34|     1|     0|   2.2381067|
+-------+------+------+------------+
only showing top 20 rows



In [11]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.6819180974180867


In [12]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [13]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|     11|    11|
|     12|    11|
|     21|    11|
|     22|    11|
|     30|    11|
|     32|    11|
|     40|    11|
|     41|    11|
|     45|    11|
|     64|    11|
|     71|    11|
|     77|    11|
|     88|    11|
|     90|    11|
|     97|    11|
+-------+------+



In [14]:
reccomendations = model.transform(single_user)

In [15]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     77|    11|   5.436677|
|     12|    11|  3.8874624|
|      0|    11|  3.1496341|
|     32|    11|  2.7830086|
|     90|    11|  2.5719254|
|     40|    11|    2.22368|
|     88|    11|  2.0796795|
|     64|    11|  1.5522953|
|     21|    11|  1.4722469|
|     45|    11|  1.4489539|
|     71|    11|  0.8689847|
|     97|    11| 0.47176176|
|     11|    11| 0.28994656|
|     22|    11|-0.63451916|
|     41|    11| -0.9962452|
|     30|    11| -1.2148972|
+-------+------+-----------+

